### Use this cell to make installations

In [4]:
!pip install pocketsphinx
!pip install pyttsx3

### Use this cell to import any libraries

In [27]:
import speech_recognition as sr
from openai import OpenAI #Only for testing purposes
import os
import openai
from dotenv import load_dotenv, find_dotenv
from playsound import playsound

### Fetching the API Key and selecting the LLM Model

In [28]:
# Load environment variables from .env file
_ = load_dotenv(find_dotenv())

# Get OpenAI API key from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    raise ValueError("OpenAI API key is not set. Please set the 'OPENAI_API_KEY' environment variable in your .env file.")

# Account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"


### Main Code Cell
#### Speech to Text using `PocketSphinx`
#### GPT Model: `gpt-3.5-turbo`

In [51]:
recognizer = sr.Recognizer()

def get_API():
    # Load environment variables from .env file
    _ = load_dotenv(find_dotenv())

    # Get OpenAI API key from environment variables
    openai_api_key = os.getenv('OPENAI_API_KEY')
    if not openai_api_key:
        raise ValueError("OpenAI API key is not set. Please set the 'OPENAI_API_KEY' environment variable in your .env file.")
    return openai_api_key

def get_Model():
    # Get the current date
    current_date = datetime.datetime.now().date()

    # Define the date after which the model should be set to "gpt-3.5-turbo"
    target_date = datetime.date(2024, 6, 12)

    # Set the model variable based on the current date
    if current_date > target_date:
        llm_model = "gpt-3.5-turbo"
    else:
        llm_model = "gpt-3.5-turbo-0301"
    return llm_model

def speech_to_text():
    '''
    1. Listens for speech after adjusting for surroundings
    2. Recognizes speech and stores it as a string
    3. Returns the string
    '''
    try:
        with sr.Microphone() as source:
            print('Adjusting for ambience...')
            recognizer.adjust_for_ambient_noise(source, duration=1)  # Adjust based on your ambient noise

            print('Listening...')
            audio = recognizer.listen(source)  
            
            print('Recognizing...')
            text = recognizer.recognize_sphinx(audio)
        return text
    except sr.WaitTimeoutError:
        print("No speech was detected in a reasonable amount of time.")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from the recognition service; {e}")
        return None
    except sr.UnknownValueError:
        print("Unable to recognize any speech.")
        return None

def send_prompt():
    try:
        with sr.Microphone() as source:
            print('Adjusting for ambience...')
            recognizer.adjust_for_ambient_noise(source, duration=1)  # Adjust based on your ambient noise

            print('Listening...')
            audio = recognizer.listen(source, 5)  
            
            print('Recognizing...')
            text = recognizer.recognize_sphinx(audio)
        return text
    except sr.WaitTimeoutError:
        print("No speech was detected in a reasonable amount of time.")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from the recognition service; {e}")
        return None
    except sr.UnknownValueError:
        print("Unable to recognize any speech.")
        return None
    
def text_to_speech(text):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=text
    )
    file_path = 'response_voice.mp3'
    response.stream_to_file(file_path)
    play_audio(file_path)
    
def write_to_file(text):
    '''
    1. Write the text to a file
    2. Returns the file path
    '''
    with open('text.txt','w') as file:
        file.write(text)
    return os.getcwd() + '/text.txt'

def read_from_file(file_path):
    with open(file_path, 'r', encoding = 'utf-8') as file:
        text = file.read()
    return text

def play_audio(file_path):
    playsound(file_path)

def process_prompt(conversation_text):
    intro_path = os.path.join(os.getcwd(), 'intro', 'intro_voice.mp3')
    play_audio(intro_path)
    prompt = send_prompt()
    print("Recognized Prompt:", prompt)  # Testing
    if prompt:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a memory assistant, listening to my conversations."},
                {"role": "user", "content": conversation_text},  # Provide the conversation context
                {"role": "user", "content": prompt}  # User's question
            ]
        )
        
        response_text = response.choices[0].message.content
        text_to_speech(response_text)
        
api = get_API()
model = get_Model()
text = read_from_file('testing/test_file.txt')
process_prompt(text)

Adjusting for ambience...
Listening...
Recognizing...
Recognized Prompt: where was might last week


/var/folders/zb/2519d8rs78v6ckcybj75fnl80000gp/T/ipykernel_1762/3638583481.py:58: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(file_path)
